In [4]:
import pandas as pd

#read csv files
df_f=pd.read_csv("follows.csv")
df_i=pd.read_csv("interests.csv")

df_f.columns = df_f.columns.str.strip()
df_i.columns = df_i.columns.str.strip()

#one-hot coding
df_i = pd.get_dummies(df_i,columns=['category'])
df_i = df_i.groupby('user_id').sum()

df_f = pd.get_dummies(df_f,columns=['followee_id'])
df_f = df_f.groupby('follower_id').sum()
df_f.index.names = ['user_id']

In [5]:
#rename df_f columns
df_f_cols= [df_f.columns.tolist()[i].strip('followee_id_') for i in range(len(df_f.columns.tolist()))]
df_f.columns= df_f_cols

In [6]:
#inner join two make both dataframes have same user_ids
df_test1 = pd.DataFrame.join(df_i,df_f,on='user_id',how='inner')
df_test1.shape

(2237, 4478)

In [7]:
#transpose datasets
df_reduced_i = df_test1.T.iloc[:883].T
df_reduced_f = df_test1.T.iloc[883:].T

In [8]:
#transpose datasets
df_f_t = df_reduced_f.T
df_f_t.index.names=['followee_id']
df_f_t.index=df_f_t.index.astype('int64')
df_reduced_i.index.names = ['followee_id']

In [9]:
#make followees are same as user_ids
df_test2 = pd.DataFrame.merge(df_reduced_i,df_f_t,left_index=True, right_index=True)
df_test2.index.names=['user_id']
df_test2.shape

(2237, 3120)

In [10]:
#transpose back to origin dataframes
df_interest = df_test2.T.iloc[:883].T
df_followee = df_test2.T.iloc[883:].T

In [11]:
#create a default df for probability
df_prob = pd.DataFrame(columns =df_followee.columns.tolist() ,index =df_followee.index.tolist() )

In [19]:
import numpy as np
from sklearn.naive_bayes import BernoulliNB

# for each followee, train a model
for i in range(2237):
    clf = BernoulliNB()
    clf.fit(df_interest, df_followee.iloc[:,i])
    
    # for each follower, predict if he/she will follow this followee
    for n in range(2237):
        #prob of 1
        prob = clf.predict_proba(np.array(df_interest.iloc[n]).reshape(1,-1))[:, 1]
        df_prob.at[df_followee.index[n], df_followee.index[i]] = prob


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [20]:
#create predicition matrix for recommended followees
df_recom_nb = pd.DataFrame(columns =df_followee.columns.tolist(), index =df_followee.index.tolist())

In [36]:
user_id = df_interest.index.values


for m in range(2237):
    
    #create a probability dataframe to sort out the foloowees with higher probability
    prob_m = df_prob.iloc[m]
    data_nb = {'user_id': user_id, 'prob': prob_m}
    df_nb = pd.DataFrame(data=data_nb)
    df_nb_sort = df_nb.sort_values(by=['prob'], axis=0, ascending=False)
    
    #generate the recommendation followees for each id
    for n in range(2237):
        nrecommendation = df_followee.iloc[n].sum()
        recom_nb_id = df_nb_sort.user_id.values.tolist()[:nrecommendation]
        df_recom_nb.at[user_id[n], recom_nb_id] = 1



In [37]:
from sklearn.metrics import precision_score
from statistics import mean

#precision score function
def prec_score(df_true, df_prediction): 
    pre = [precision_score(df_true.loc[i],df_prediction.loc[i])for i in df_prediction.index.values ]
    return mean(pre)

In [38]:
from sklearn.metrics import f1_score

#f1 score function
def fone_score(df_true, df_prediction):
    f1 = [f1_score(df_true.loc[i],df_prediction.loc[i]) for i in df_prediction.index.values]
    return mean(f1)

In [39]:
#prediction/ recommended followee
df_pred = df_recom_nb.fillna(0)
df_true = df_followee

In [40]:
print("precision_score is: ",prec_score(df_true,df_pred))
print("f1_score is: ",fone_score(df_true,df_pred))

precision_score is:  0.5707878592168595
f1_score is:  0.6061809129037916
